In [8]:
import pandas as pd
import json
import glob, os
import requests
from datetime import datetime
import hashlib

%load_ext autoreload
%autoreload 2

from dotenv import load_dotenv
load_dotenv('.env')

URL = os.getenv("KG_URL_FCT")
TOK3N = os.getenv("KG_TOKEN")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
import importlib.util
import sys
spec = importlib.util.spec_from_file_location("llm", "../../../utils/llm.py")
h = importlib.util.module_from_spec(spec)
sys.modules["llm"] = h
spec.loader.exec_module(h)

In [10]:
def ask(CONTEXT,QUESTION,prefix,functions=h.functionsT35,overwrite=False,MODEL="gpt-3.5-turbo-1106",seed=""):
    REQ = {
        "context": CONTEXT,
        "question": QUESTION,
        "model": MODEL,
        "functions": functions,
        "token": TOK3N,
        "overwrite": overwrite,
        "source": "local-risksT3.5",
        "seed" : seed
    }

    H = h.hashme(CONTEXT+QUESTION+seed)
    cached = "cache/"+prefix+"_"+H+".json"
    if not os.path.isfile(cached) or overwrite:
        x = requests.post(URL+"fct/", json = REQ)
        answer = json.loads(x.text)["messages"][-1]
        h.svt(cached,answer)
    else:
        answer = h.ldt(cached)
    
    return answer

In [11]:
from langchain.vectorstores import Chroma
vectordb = Chroma(persist_directory="../../../bok/data/vectorDB/")

DOCS  = list(vectordb.get()["documents"])
SRC = [x["article"] for x in vectordb.get()["metadatas"]]
print(len(DOCS),len(SRC))


6643 6643


In [12]:
import glob, os
x = glob.glob("cache/*.json")
x = [y.split(os.sep)[-1].split(".")[0][5:] for y in x]
len(x),x[0]

(6617, '0006ad97768785a44e1e31a9f8b06596')

In [13]:
RISKS = []
for k in range(len(SRC)):
    HASH = h.hashme(str(DOCS[k]))
    if HASH not in x:
        if len(str(DOCS[k])) > 100:
            done = False
            while not done:
                for seed in ["","1","2","3"]:
                    #print(str(SRC[k]),str(DOCS[k]))
                    try:
                        answer = ask("Do a risk assessment of the following text - list as many risks as possible, ideally at least 3.",str(DOCS[k]),prefix="risk",functions=h.functionsT35,overwrite=False,seed=seed)
                        D = pd.DataFrame(json.loads(answer["function_call"]["arguments"])["risks"])
                        D["src"] = str(SRC[k])
                        D["hash"] = HASH
                        if not len(D.columns) == 8:
                            print("Must redo")
                            answer = ask("Do a risk assessment of the following text - list as many risks as possible, ideally at least 3.",str(DOCS[k]),prefix="risk",functions=h.functionsT35,overwrite=True,seed=seed)
                            done = False
                        else:
                            done = True
                    except:
                        done = True
        RISKS.append(D)
        if not k%10:
            print(k)
answer

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1880
1890
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990
2000
2010
2020
2030
2040
2050
2060
2070
2080
2090
2100
2110
2120
2130
2140
2150
2160
2170
2180
2190
2200
2210
2

{'role': 'assistant',
 'content': None,
 'function_call': {'name': 'get_risks',
  'arguments': '{"risks":[{"Description":"Healthcare and nursing home workers testing positive for COVID-19","Impact":"Increased risk of infection and transmission in healthcare and nursing home settings","Mitigation":"Implement strict infection prevention and control (IPC) measures, provide adequate PPE, and conduct regular testing and surveillance","Technologies":"Advanced PPE, rapid testing technologies, and contact tracing apps","People":"Healthcare workers, nursing home staff, and residents","Owner":"Healthcare administrators, facility managers, and government health agencies"},{"Description":"Heterogeneity in COVID-19 positivity based on study region","Impact":"Differences in infection rates and transmission dynamics between regions","Mitigation":"Tailoring IPC measures to specific regional needs, monitoring regional trends, and sharing best practices between regions","Technologies":"Data analytics fo

In [19]:
dfBib = pd.read_parquet("../../../bok/data/bibliography.parquet.gzip")
dfBib= dfBib.reset_index()
dfBib.columns = ["src","author","title","doi"]
dfBib

,src,author,title,doi
0,aguero_covid-19_2021,"Agüero, Jorge M.",{COVID}-19 and the rise of intimate partner vi...,10.1016/j.worlddev.2020.105217
1,gurney_dangerous_2021,"Gurney, Craig M.",Dangerous liaisons? {Applying} the social harm...,10.1080/19491247.2021.1971033
2,olivier_infrastructures_2022,"Olivier, Jéan-Louise and Mee, Kathleen and Pow...",Infrastructures of {Care} for {Public} {Housin...,10.1080/08111146.2022.2123317
3,allam_emergence_2019,"Allam, Zaheer",The {Emergence} of {Anti}-{Privacy} and {Contr...,10.3390/smartcities2010007
4,serafini_psychological_2020,"Serafini, G and Parmigiani, B and Amerio, A an...",The psychological impact of {COVID}-19 on the ...,10.1093/qjmed/hcaa201
...,...,...,...,...
377,fagerholm_analysis_2022,"Fagerholm, Nora and Samuelsson, Karl and Eilol...",Analysis of pandemic outdoor recreation and gr...,10.1038/s42949-022-00068-8
378,cheng_potential_2023,"Cheng, Kunming and Li, Zhiyong and He, Yongbin...",Potential {Use} of {Artificial} {Intelligence}...,10.1007/s10439-023-03203-3
379,ayers_comparing_2023,"Ayers, John W. and Poliak, Adam and Dredze, Ma...",Comparing {Physician} and {Artificial} {Intell...,10.1001/jamainternmed.2023.1838
380,pruszynski_phenomenon_2023,"Pruszyński, Jacek and Kocik, Janusz and Pruszy...",The phenomenon of urbanisation from a public h...,10.12775/JEHS.2023.13.04.004


In [52]:
import regex

def is_hangul(value):
    if regex.search(r'\p{IsHangul}', value):
        return True
    return False

In [71]:
dfRisks = pd.concat(RISKS).reset_index(drop=True)
for item in ["Technologies","People","Owner"]:
    dfRisks[item] = dfRisks[item].apply(lambda x: [y.strip().capitalize() for y in str(x).strip(".").split(",")])
dfRisks

,Description,Impact,Mitigation,Technologies,People,Owner,src,hash
0,Zoonotic infections from domestic and wild ani...,Severe economic disruption and disaster if not...,Improved animal-to-human interaction managemen...,"[Dna sequencing for pathogen identification, D...","[General population, Healthcare workers, Trave...","[Healthcare systems, Governments, Animal contr...",abideen_mitigation_2020,f5f5af1eef7fe66226065027d070a105
1,"Spread risk based on population, poverty level...",Potential for localized outbreaks due to mutat...,Improved surveillance and control of environme...,[Data analytics for monitoring population move...,"[General population, Travelers, Individuals in...","[Healthcare systems, Governments, Environmenta...",abideen_mitigation_2020,f5f5af1eef7fe66226065027d070a105
2,"Spread of contagious diseases like influenza, ...","High mortality rates, strain on healthcare sys...","Enhanced hygiene protocols, rapid response pla...","[Vaccines, Antiviral drugs, Contact tracing apps]","[Global population, Healthcare workers, Travel...","[Governments, Healthcare organizations, Intern...",abideen_mitigation_2020,e2fefe64d3f6f552fd8c225341c476cb
3,"Failure in containing outbreaks, political mis...","Increased spread of disease, misinformation, l...","Transparent communication, effective leadershi...","[Information dissemination tools, Crisis manag...","[General population, Government officials, Med...","[Government, Media organizations, Public healt...",abideen_mitigation_2020,e2fefe64d3f6f552fd8c225341c476cb
4,Illicit trade of wildlife meat and potential f...,Increased risk of zoonotic disease transmissio...,"Regulation of wildlife trade, enforcement of f...","[Dna testing for wildlife species, Monitoring ...","[Wildlife traders, Public health agencies, Res...","[Government authorities, Wildlife conservation...",abideen_mitigation_2020,e2fefe64d3f6f552fd8c225341c476cb
...,...,...,...,...,...,...,...,...
18182,Testing alone was not enough to reduce R0 belo...,Inability to reduce R0 below 1 solely through ...,Combination of testing with high-quality infec...,"[Testing technologies, Infection control techn...","[General population, Healthcare workers]","[Public health authorities, Healthcare adminis...",ingram_covid-19_2021,ddc792c4fd824ca02fa7d4432994822d
18183,The effectiveness of test and trace depends st...,Ineffective contact tracing can lead to uncont...,Improving coverage and timeliness of contact t...,"[Contact tracing technologies, Data management...",[Individuals who require contact tracing],"[Public health authorities, Contact tracing or...",ingram_covid-19_2021,ddc792c4fd824ca02fa7d4432994822d
18184,Healthcare and nursing home workers testing po...,Increased risk of infection and transmission i...,Implement strict infection prevention and cont...,"[Advanced ppe, Rapid testing technologies, And...","[Healthcare workers, Nursing home staff, And r...","[Healthcare administrators, Facility managers,...",ingram_covid-19_2021,3f594f177c862efe64ddde788d595864
18185,Heterogeneity in COVID-19 positivity based on ...,Differences in infection rates and transmissio...,Tailoring IPC measures to specific regional ne...,[Data analytics for regional surveillance and ...,"[Public health officials, Researchers, And pol...","[Public health agencies, Research institutions...",ingram_covid-19_2021,3f594f177c862efe64ddde788d595864


In [72]:
dfRisks = pd.concat(RISKS).reset_index(drop=True)
dfRisks = dfRisks.merge(dfBib, on="src",how="left")
dfRisks = dfRisks.dropna().reset_index(drop=True)
for item in ["Technologies","People","Owner"]:
    dfRisks[item] = dfRisks[item].apply(lambda x: [y.capitalize().strip() for y in str(x).strip(".").split(",")])
dfRisks["is_hangul"] = dfRisks.Description.apply(lambda x: is_hangul(x))
dfRisks = dfRisks[~dfRisks.is_hangul].drop(columns=["is_hangul"])
dfRisks = dfRisks.dropna().reset_index(drop=True)
dfRisks.to_parquet("newrisks.parquet.gzip",compression="gzip")
dfRisks

,Description,Impact,Mitigation,Technologies,People,Owner,src,hash,author,title,doi
0,Zoonotic infections from domestic and wild ani...,Severe economic disruption and disaster if not...,Improved animal-to-human interaction managemen...,"[Dna sequencing for pathogen identification, d...","[General population, healthcare workers, trave...","[Healthcare systems, governments, animal contr...",abideen_mitigation_2020,f5f5af1eef7fe66226065027d070a105,"Abideen, Ahmed Zainul and Mohamad, Fazeeda Bin...",Mitigation strategies to fight the {COVID}-19 ...,10.1108/jhr-04-2020-0109
1,"Spread risk based on population, poverty level...",Potential for localized outbreaks due to mutat...,Improved surveillance and control of environme...,[Data analytics for monitoring population move...,"[General population, travelers, individuals in...","[Healthcare systems, governments, environmenta...",abideen_mitigation_2020,f5f5af1eef7fe66226065027d070a105,"Abideen, Ahmed Zainul and Mohamad, Fazeeda Bin...",Mitigation strategies to fight the {COVID}-19 ...,10.1108/jhr-04-2020-0109
2,"Spread of contagious diseases like influenza, ...","High mortality rates, strain on healthcare sys...","Enhanced hygiene protocols, rapid response pla...","[Vaccines, antiviral drugs, contact tracing apps]","[Global population, healthcare workers, travel...","[Governments, healthcare organizations, intern...",abideen_mitigation_2020,e2fefe64d3f6f552fd8c225341c476cb,"Abideen, Ahmed Zainul and Mohamad, Fazeeda Bin...",Mitigation strategies to fight the {COVID}-19 ...,10.1108/jhr-04-2020-0109
3,"Failure in containing outbreaks, political mis...","Increased spread of disease, misinformation, l...","Transparent communication, effective leadershi...","[Information dissemination tools, crisis manag...","[General population, government officials, med...","[Government, media organizations, public healt...",abideen_mitigation_2020,e2fefe64d3f6f552fd8c225341c476cb,"Abideen, Ahmed Zainul and Mohamad, Fazeeda Bin...",Mitigation strategies to fight the {COVID}-19 ...,10.1108/jhr-04-2020-0109
4,Illicit trade of wildlife meat and potential f...,Increased risk of zoonotic disease transmissio...,"Regulation of wildlife trade, enforcement of f...","[Dna testing for wildlife species, monitoring ...","[Wildlife traders, public health agencies, res...","[Government authorities, wildlife conservation...",abideen_mitigation_2020,e2fefe64d3f6f552fd8c225341c476cb,"Abideen, Ahmed Zainul and Mohamad, Fazeeda Bin...",Mitigation strategies to fight the {COVID}-19 ...,10.1108/jhr-04-2020-0109
...,...,...,...,...,...,...,...,...,...,...,...
18149,Testing alone was not enough to reduce R0 belo...,Inability to reduce R0 below 1 solely through ...,Combination of testing with high-quality infec...,"[Testing technologies, infection control techn...","[General population, healthcare workers]","[Public health authorities, healthcare adminis...",ingram_covid-19_2021,ddc792c4fd824ca02fa7d4432994822d,"Ingram, Carolyn and Downey, Vicky and Roe, Mar...",{COVID}-19 {Prevention} and {Control} {Measure...,10.3390/ijerph18157847
18150,The effectiveness of test and trace depends st...,Ineffective contact tracing can lead to uncont...,Improving coverage and timeliness of contact t...,"[Contact tracing technologies, data management...",[Individuals who require contact tracing],"[Public health authorities, contact tracing or...",ingram_covid-19_2021,ddc792c4fd824ca02fa7d4432994822d,"Ingram, Carolyn and Downey, Vicky and Roe, Mar...",{COVID}-19 {Prevention} and {Control} {Measure...,10.3390/ijerph18157847
18151,Healthcare and nursing home workers testing po...,Increased risk of infection and transmission i...,Implement strict infection prevention and cont...,"[Advanced ppe, rapid testing technologies, and...","[Healthcare workers, nursing home staff, and r...","[Healthcare administrators, facility managers,...",ingram_covid-19_2021,3f594f177c862efe64ddde788d595864,"Ingram, Carolyn and Downey, Vicky and Roe, Mar...",{COVID}-19 {Pr

# Reviews

In [62]:
# Reviews
df = pd.read_parquet("newrisks.parquet.gzip")
df.head(3)

,Description,Impact,Mitigation,Technologies,People,Owner,src,hash,author,title,doi
0,Zoonotic infections from domestic and wild ani...,Severe economic disruption and disaster if not...,Improved animal-to-human interaction managemen...,"[Dna sequencing for pathogen identification, d...","[General population, healthcare workers, trave...","[Healthcare systems, governments, animal contr...",abideen_mitigation_2020,f5f5af1eef7fe66226065027d070a105,"Abideen, Ahmed Zainul and Mohamad, Fazeeda Bin...",Mitigation strategies to fight the {COVID}-19 ...,10.1108/jhr-04-2020-0109
1,"Spread risk based on population, poverty level...",Potential for localized outbreaks due to mutat...,Improved surveillance and control of environme...,[Data analytics for monitoring population move...,"[General population, travelers, individuals in...","[Healthcare systems, governments, environmenta...",abideen_mitigation_2020,f5f5af1eef7fe66226065027d070a105,"Abideen, Ahmed Zainul and Mohamad, Fazeeda Bin...",Mitigation strategies to fight the {COVID}-19 ...,10.1108/jhr-04-2020-0109
2,"Spread of contagious diseases like influenza, ...","High mortality rates, strain on healthcare sys...","Enhanced hygiene protocols, rapid response pla...","[Vaccines, antiviral drugs, contact tracing apps]","[Global population, healthcare workers, travel...","[Governments, healthcare organizations, intern...",abideen_mitigation_2020,e2fefe64d3f6f552fd8c225341c476cb,"Abideen, Ahmed Zainul and Mohamad, Fazeeda Bin...",Mitigation strategies to fight the {COVID}-19 ...,10.1108/jhr-04-2020-0109
